# NFL Games in 2010

This notebook explores the games of the 2010 NFL season.

## Import the data

The data is defined using RexlScript in the text file `data/NFL-2010-Games.rexl`.
The data could also come from a parquet file or rbin file.

In [1]:
import "../data/NFL-2010-Games.rexl";

In [2]:
#!globals

Variable,Type
Games,"{D:d, H:s, HP:i8, V:s, VP:i8, W:i8}*"


The script introduced one _global_ named `Games` with the indicated type.
The `{...}` part of the type indicates a _record_ with the indicated field names and types.
The `*` indicates the type is _sequence of_ the record type, also called a table type.

Consequently, `Games` is a table (sequence of record) with columns:
* `D`: the date the game was played, of type `Date`, represented by `d`.
* `H`: the name of the home team, of type `Text`, represented by `s` (for string).
* `HP`: the number of points scored by the home team, of type `i8`, an 8-byte signed integer value.
* `V`: the name of the visiting team, of type `Text`.
* `VP`: the number of points scored by the visiting team, of type `i8`.
* `W`: the week number the game was played, of type `i8`.

## Basics

Show the first 5 games.

In [3]:
Take(Games, 5)

Seq<{Date,str,i8,str,i8,i8}>
   0) { D: 2010/09/09, H: NO_, HP: 14, V: MIN, VP: 9, W: 1 }
   1) { D: 2010/09/12, H: TB_, HP: 17, V: CLE, VP: 14, W: 1 }
   2) { D: 2010/09/12, H: BUF, HP: 10, V: MIA, VP: 15, W: 1 }
   3) { D: 2010/09/12, H: NE_, HP: 38, V: CIN, VP: 24, W: 1 }
   4) { D: 2010/09/12, H: JAC, HP: 24, V: DEN, VP: 17, W: 1 }


The pipe syntax is often convenient. The left operand of `->` is treated as the first argument
of the function invocation.

In [4]:
Games->Take(5)

Seq<{Date,str,i8,str,i8,i8}>
   0) { D: 2010/09/09, H: NO_, HP: 14, V: MIN, VP: 9, W: 1 }
   1) { D: 2010/09/12, H: TB_, HP: 17, V: CLE, VP: 14, W: 1 }
   2) { D: 2010/09/12, H: BUF, HP: 10, V: MIA, VP: 15, W: 1 }
   3) { D: 2010/09/12, H: NE_, HP: 38, V: CIN, VP: 24, W: 1 }
   4) { D: 2010/09/12, H: JAC, HP: 24, V: DEN, VP: 17, W: 1 }


Show the games Seattle played, both as home team and as visiting team.

Note: statements are separated by semicolon.

In [5]:
Games->TakeIf(H = "SEA");
Games->TakeIf(V = "SEA")

Seq<{Date,str,i8,str,i8,i8}>
   0) { D: 2010/09/12, H: SEA, HP: 31, V: SF_, VP: 6, W: 1 }
   1) { D: 2010/09/26, H: SEA, HP: 27, V: SD_, VP: 20, W: 3 }
   2) { D: 2010/10/24, H: SEA, HP: 22, V: ARI, VP: 10, W: 7 }
   3) { D: 2010/11/07, H: SEA, HP: 7, V: NYG, VP: 41, W: 9 }
   4) { D: 2010/11/28, H: SEA, HP: 24, V: KC_, VP: 42, W: 12 }
   5) { D: 2010/12/05, H: SEA, HP: 31, V: CAR, VP: 14, W: 13 }
   6) { D: 2010/12/19, H: SEA, HP: 18, V: ATL, VP: 34, W: 15 }
   7) { D: 2011/01/02, H: SEA, HP: 16, V: STL, VP: 6, W: 17 }
Seq<{Date,str,i8,str,i8,i8}>
   0) { D: 2010/09/19, H: DEN, HP: 31, V: SEA, VP: 14, W: 2 }
   1) { D: 2010/10/03, H: STL, HP: 20, V: SEA, VP: 3, W: 4 }
   2) { D: 2010/10/17, H: CHI, HP: 20, V: SEA, VP: 23, W: 6 }
   3) { D: 2010/10/31, H: OAK, HP: 33, V: SEA, VP: 3, W: 8 }
   4) { D: 2010/11/14, H: ARI, HP: 18, V: SEA, VP: 36, W: 10 }
   5) { D: 2010/11/21, H: NO_, HP: 34, V: SEA, VP: 19, W: 11 }
   6) { D: 2010/12/12, H: SF_, HP: 40, V: SEA, VP: 21, W: 14 }
   7) { D:

How many games were played?

In [6]:
Games->Count()

256


How many games were won by the home team?

In [7]:
Games->Count(HP > VP)

143


What percentage of games were won by the home team?

In [8]:
Games->Count(HP > VP) / Games->Count() * 100

55.859375


How many tie games?

In [9]:
Games->Count(VP = HP)

0


How many games were decided by less than 7 points?

In [10]:
Games->Count(Abs(HP - VP) < 7)

103


## AllGames

We want to build a `Teams` table with columns for the team name and various statistics, and a column containing
a nested table of the games played by the team.

Start with a table, `AllGames`, with columns for team `A` and team `B`, where each game is represented twice,
once from the perspective of each team. The _augmenting record projection operator_ `+>` can add, rename, and
drop fields. The `Chain` function chains together two sequences of the same item type.

Also sort by the date and team `A`. The default sort direction is _up_ for text and _down_ for all other types,
including the `Date` type. In this case we want to sort _up_ for both. We could either use the `SortUp` function
or, as we've done here, use the `[<]` _directive_ for the `D` column.

In [11]:
AllGames :=
    Chain(
        Games+>{ A: H, B: V, AP: HP, BP: VP },
        Games+>{ A: V, B: H, AP: VP, BP: HP })
    ->Sort([<] D, A)

Show all records for the first week of the season. Note that the first two records are for the same game, but with the teams
(Minnesota and New Orleans) swapped. Similarly, all records have a matching record where the teams are swapped.

In [12]:
AllGames->TakeIf(W = 1)

Seq<{str,i8,str,i8,Date,i8}>
   0) { A: MIN, AP: 9, B: NO_, BP: 14, D: 2010/09/09, W: 1 }
   1) { A: NO_, AP: 14, B: MIN, BP: 9, D: 2010/09/09, W: 1 }
   2) { A: ARI, AP: 17, B: STL, BP: 13, D: 2010/09/12, W: 1 }
   3) { A: ATL, AP: 9, B: PIT, BP: 15, D: 2010/09/12, W: 1 }
   4) { A: BUF, AP: 10, B: MIA, BP: 15, D: 2010/09/12, W: 1 }
   5) { A: CAR, AP: 18, B: NYG, BP: 31, D: 2010/09/12, W: 1 }
   6) { A: CHI, AP: 19, B: DET, BP: 14, D: 2010/09/12, W: 1 }
   7) { A: CIN, AP: 24, B: NE_, BP: 38, D: 2010/09/12, W: 1 }
   8) { A: CLE, AP: 14, B: TB_, BP: 17, D: 2010/09/12, W: 1 }
   9) { A: DAL, AP: 7, B: WAS, BP: 13, D: 2010/09/12, W: 1 }
  10) { A: DEN, AP: 17, B: JAC, BP: 24, D: 2010/09/12, W: 1 }
  11) { A: DET, AP: 14, B: CHI, BP: 19, D: 2010/09/12, W: 1 }
  12) { A: GB_, AP: 27, B: PHI, BP: 20, D: 2010/09/12, W: 1 }
  13) { A: HOU, AP: 34, B: IND, BP: 24, D: 2010/09/12, W: 1 }
  14) { A: IND, AP: 24, B: HOU, BP: 34, D: 2010/09/12, W: 1 }
  15) { A: JAC, AP: 24, B: DEN, BP: 17, D: 20

Show all games for Seattle.

In [13]:
AllGames->TakeIf(A = "SEA")

Seq<{str,i8,str,i8,Date,i8}>
   0) { A: SEA, AP: 31, B: SF_, BP: 6, D: 2010/09/12, W: 1 }
   1) { A: SEA, AP: 14, B: DEN, BP: 31, D: 2010/09/19, W: 2 }
   2) { A: SEA, AP: 27, B: SD_, BP: 20, D: 2010/09/26, W: 3 }
   3) { A: SEA, AP: 3, B: STL, BP: 20, D: 2010/10/03, W: 4 }
   4) { A: SEA, AP: 23, B: CHI, BP: 20, D: 2010/10/17, W: 6 }
   5) { A: SEA, AP: 22, B: ARI, BP: 10, D: 2010/10/24, W: 7 }
   6) { A: SEA, AP: 3, B: OAK, BP: 33, D: 2010/10/31, W: 8 }
   7) { A: SEA, AP: 7, B: NYG, BP: 41, D: 2010/11/07, W: 9 }
   8) { A: SEA, AP: 36, B: ARI, BP: 18, D: 2010/11/14, W: 10 }
   9) { A: SEA, AP: 19, B: NO_, BP: 34, D: 2010/11/21, W: 11 }
  10) { A: SEA, AP: 24, B: KC_, BP: 42, D: 2010/11/28, W: 12 }
  11) { A: SEA, AP: 31, B: CAR, BP: 14, D: 2010/12/05, W: 13 }
  12) { A: SEA, AP: 21, B: SF_, BP: 40, D: 2010/12/12, W: 14 }
  13) { A: SEA, AP: 18, B: ATL, BP: 34, D: 2010/12/19, W: 15 }
  14) { A: SEA, AP: 15, B: TB_, BP: 38, D: 2010/12/26, W: 16 }
  15) { A: SEA, AP: 16, B: STL, BP: 6,

## Teams

Define `Teams` to be a grouping of `AllGames`.

The `GroupBy` function is very flexible and powerful. Its usage here is fairly simple. The first
argument specifies the grouping key to be the name of team `A` and specifies that in the
_group record_, this value should be a field with name `Name`. The second argument specifies
the name of the auto-generated nested table field in the group record to be `XGames`. We use
the prefix `X` just so it is listed last in displays.

We also sort by team `Name`.

In [14]:
Teams :=
    AllGames
    ->GroupBy(Name: A, XGames)
    ->Sort(Name)

Show the type of the `Teams` table. Note that the type of the `XGames` column is a table type (sequence
of record type). Also note that the record type for `XGames` does not have the `A` field. Since field `A`
was used as a grouping key and promoted to the group record as a field (in this case renamed to `Name`),
`A` was dropped from the nested table.

In [15]:
Teams->GetType()

{Name:s, XGames:{AP:i8, B:s, BP:i8, D:d, W:i8}*}*


Show the first 2 teams.

In [16]:
Teams->Take(2)

Seq<{str,Seq<{i8,str,i8,Date,i8}>}>
   0) { Name: ARI, 
        XGames: Seq<{i8,str,i8,Date,i8}>
             0) { AP: 17, B: STL, BP: 13, D: 2010/09/12, W: 1 }
             1) { AP: 7, B: ATL, BP: 41, D: 2010/09/19, W: 2 }
             2) { AP: 24, B: OAK, BP: 23, D: 2010/09/26, W: 3 }
             3) { AP: 10, B: SD_, BP: 41, D: 2010/10/03, W: 4 }
             4) { AP: 30, B: NO_, BP: 20, D: 2010/10/10, W: 5 }
             5) { AP: 10, B: SEA, BP: 22, D: 2010/10/24, W: 7 }
             6) { AP: 35, B: TB_, BP: 38, D: 2010/10/31, W: 8 }
             7) { AP: 24, B: MIN, BP: 27, D: 2010/11/07, W: 9 }
             8) { AP: 18, B: SEA, BP: 36, D: 2010/11/14, W: 10 }
             9) { AP: 13, B: KC_, BP: 31, D: 2010/11/21, W: 11 }
            10) { AP: 6, B: SF_, BP: 27, D: 2010/11/29, W: 12 }
            11) { AP: 6, B: STL, BP: 19, D: 2010/12/05, W: 13 }
            12) { AP: 43, B: DEN, BP: 13, D: 2010/12/12, W: 14 }
            13) { AP: 12, B: CAR, BP: 19, D: 2010/12/19, W: 15 }
    

Redefine `Teams`, adding some basic statistics. Note the use of the `[key]`, `[group]` and `[auto]` directives.

The `[group]` directive indicates an aggregation, where the items of a group are available (as a sequence) via
the `group` variable. In this case, we compute:
* The size of the group (number of games played) and give that field the name `Number`.
* The number of games won by the team (the `Wins` field).
* The total number of points scored by the team (the `TotAP` field).
* The total number of points scored by opponents (the `TotBP` field).
* The total point differential (the `TotDiff` field).

In [17]:
Teams :=
    AllGames
    ->GroupBy(
        [key] Name: A,
        [group] Number: group->Count(),
        [group] Wins: group->Count(AP > BP),
        [group] TotAP: group->Sum(AP),
        [group] TotBP: group->Sum(BP),
        [group] TotDiff: group->Sum(AP - BP),
        [auto] XGames)
    ->Sort(Name)

Show the type.

In [18]:
Teams->GetType()

{Name:s, Number:i8, TotAP:i8, TotBP:i8, TotDiff:i8, Wins:i8, XGames:{AP:i8, B:s, BP:i8, D:d, W:i8}*}*


Show the first two teams.

In [19]:
Teams->Take(2)

Seq<{str,i8,i8,i8,i8,i8,Seq<{i8,str,i8,Date,i8}>}>
   0) { Name: ARI, Number: 16, TotAP: 289, TotBP: 434, TotDiff: -145, Wins: 5, 
        XGames: Seq<{i8,str,i8,Date,i8}>
             0) { AP: 17, B: STL, BP: 13, D: 2010/09/12, W: 1 }
             1) { AP: 7, B: ATL, BP: 41, D: 2010/09/19, W: 2 }
             2) { AP: 24, B: OAK, BP: 23, D: 2010/09/26, W: 3 }
             3) { AP: 10, B: SD_, BP: 41, D: 2010/10/03, W: 4 }
             4) { AP: 30, B: NO_, BP: 20, D: 2010/10/10, W: 5 }
             5) { AP: 10, B: SEA, BP: 22, D: 2010/10/24, W: 7 }
             6) { AP: 35, B: TB_, BP: 38, D: 2010/10/31, W: 8 }
             7) { AP: 24, B: MIN, BP: 27, D: 2010/11/07, W: 9 }
             8) { AP: 18, B: SEA, BP: 36, D: 2010/11/14, W: 10 }
             9) { AP: 13, B: KC_, BP: 31, D: 2010/11/21, W: 11 }
            10) { AP: 6, B: SF_, BP: 27, D: 2010/11/29, W: 12 }
            11) { AP: 6, B: STL, BP: 19, D: 2010/12/05, W: 13 }
            12) { AP: 43, B: DEN, BP: 13, D: 2010/12/12, W:

Show all teams without the `XGames` column, sorted by multiple keys.

The _augmenting record projection operator_ `+>` drops a field when the field
name is followed by a `null` literal.

We sort by the number of wins, with tie breaker total point differential,
with further tie breaker of the team name. Note that `IND` is listed before `NYG`
because of this final tie breaker. The default sort order is _down_ for
numeric and _up_ for text.

In [20]:
Teams
    +>{ XGames: null }
    ->Sort(Wins, TotDiff, Name)

Seq<{str,i8,i8,i8,i8,i8}>
   0) { Name: NE_, Number: 16, TotAP: 518, TotBP: 313, TotDiff: 205, Wins: 14 }
   1) { Name: ATL, Number: 16, TotAP: 414, TotBP: 288, TotDiff: 126, Wins: 13 }
   2) { Name: PIT, Number: 16, TotAP: 375, TotBP: 232, TotDiff: 143, Wins: 12 }
   3) { Name: BAL, Number: 16, TotAP: 357, TotBP: 270, TotDiff: 87, Wins: 12 }
   4) { Name: NO_, Number: 16, TotAP: 384, TotBP: 307, TotDiff: 77, Wins: 11 }
   5) { Name: NYJ, Number: 16, TotAP: 367, TotBP: 304, TotDiff: 63, Wins: 11 }
   6) { Name: CHI, Number: 16, TotAP: 334, TotBP: 286, TotDiff: 48, Wins: 11 }
   7) { Name: GB_, Number: 16, TotAP: 388, TotBP: 240, TotDiff: 148, Wins: 10 }
   8) { Name: PHI, Number: 16, TotAP: 439, TotBP: 377, TotDiff: 62, Wins: 10 }
   9) { Name: IND, Number: 16, TotAP: 435, TotBP: 388, TotDiff: 47, Wins: 10 }
  10) { Name: NYG, Number: 16, TotAP: 394, TotBP: 347, TotDiff: 47, Wins: 10 }
  11) { Name: KC_, Number: 16, TotAP: 366, TotBP: 326, TotDiff: 40, Wins: 10 }
  12) { Name: TB_, Num

Define `TeamStats` to be the above.

In [21]:
TeamStats := Teams
    +>{ XGames: null }
    ->Sort(Wins, TotDiff, Name)

In [22]:
TeamStats

Seq<{str,i8,i8,i8,i8,i8}>
   0) { Name: NE_, Number: 16, TotAP: 518, TotBP: 313, TotDiff: 205, Wins: 14 }
   1) { Name: ATL, Number: 16, TotAP: 414, TotBP: 288, TotDiff: 126, Wins: 13 }
   2) { Name: PIT, Number: 16, TotAP: 375, TotBP: 232, TotDiff: 143, Wins: 12 }
   3) { Name: BAL, Number: 16, TotAP: 357, TotBP: 270, TotDiff: 87, Wins: 12 }
   4) { Name: NO_, Number: 16, TotAP: 384, TotBP: 307, TotDiff: 77, Wins: 11 }
   5) { Name: NYJ, Number: 16, TotAP: 367, TotBP: 304, TotDiff: 63, Wins: 11 }
   6) { Name: CHI, Number: 16, TotAP: 334, TotBP: 286, TotDiff: 48, Wins: 11 }
   7) { Name: GB_, Number: 16, TotAP: 388, TotBP: 240, TotDiff: 148, Wins: 10 }
   8) { Name: PHI, Number: 16, TotAP: 439, TotBP: 377, TotDiff: 62, Wins: 10 }
   9) { Name: IND, Number: 16, TotAP: 435, TotBP: 388, TotDiff: 47, Wins: 10 }
  10) { Name: NYG, Number: 16, TotAP: 394, TotBP: 347, TotDiff: 47, Wins: 10 }
  11) { Name: KC_, Number: 16, TotAP: 366, TotBP: 326, TotDiff: 40, Wins: 10 }
  12) { Name: TB_, Num

## Divisions

Now we consider which teams played each other and how many times. To start, look only at
Seattle and Arizona, group their games by opponent name, and count the number of items
in each group. Since there is no argument with the `[auto]` directive, the items of
the group are not present in a nested table.

In [23]:
Teams
    ->TakeIf(Name in [ "SEA", "ARI" ])
    ->{
        Name,
        XOpps: XGames->GroupBy([key] Opp: B, [group] Num: group->Count())
      }

Seq<{str,Seq<{i8,str}>}>
   0) { Name: ARI, 
        XOpps: Seq<{i8,str}>
             0) { Num: 2, Opp: STL }
             1) { Num: 1, Opp: ATL }
             2) { Num: 1, Opp: OAK }
             3) { Num: 1, Opp: SD_ }
             4) { Num: 1, Opp: NO_ }
             5) { Num: 2, Opp: SEA }
             6) { Num: 1, Opp: TB_ }
             7) { Num: 1, Opp: MIN }
             8) { Num: 1, Opp: KC_ }
             9) { Num: 2, Opp: SF_ }
            10) { Num: 1, Opp: DEN }
            11) { Num: 1, Opp: CAR }
            12) { Num: 1, Opp: DAL }
      }
   1) { Name: SEA, 
        XOpps: Seq<{i8,str}>
             0) { Num: 2, Opp: SF_ }
             1) { Num: 1, Opp: DEN }
             2) { Num: 1, Opp: SD_ }
             3) { Num: 2, Opp: STL }
             4) { Num: 1, Opp: CHI }
             5) { Num: 2, Opp: ARI }
             6) { Num: 1, Opp: OAK }
             7) { Num: 1, Opp: NYG }
             8) { Num: 1, Opp: NO_ }
             9) { Num: 1, Opp: KC_ }
            10) { 

Keep only rivals, that is, teams that were played more than once.

In [24]:
Teams
    ->TakeIf(Name in [ "SEA", "ARI" ])
    ->{
        Name,
        XOpps: XGames
            ->GroupBy([key] Opp: B, [group] Num: group->Count())
            ->TakeIf(Num > 1)
      }

Seq<{str,Seq<{i8,str}>}>
   0) { Name: ARI, 
        XOpps: Seq<{i8,str}>
             0) { Num: 2, Opp: STL }
             1) { Num: 2, Opp: SEA }
             2) { Num: 2, Opp: SF_ }
      }
   1) { Name: SEA, 
        XOpps: Seq<{i8,str}>
             0) { Num: 2, Opp: SF_ }
             1) { Num: 2, Opp: STL }
             2) { Num: 2, Opp: ARI }
      }


Sort the rival names, with the team name added in, and glue them together as a single text value.
Call this the team's "division".

In [25]:
Teams
    ->TakeIf(Name in [ "SEA", "ARI" ])
    ->{
        Name,
        XDiv: XGames
            ->GroupBy([key] Opp: B, [group] Num: group->Count())
            ->TakeIf(Num > 1)
            ->(Opp)
            ->Chain([ Name ])
            ->Sort()
            ->Concat("|")
      }

Seq<{str,str}>
   0) { Name: ARI, XDiv: ARI|SEA|SF_|STL }
   1) { Name: SEA, XDiv: ARI|SEA|SF_|STL }


Add `XDiv` to all teams.

In [26]:
TeamsWithDiv :=
    Teams+>
    {
        XDiv: XGames
            ->GroupBy([key] Opp: B, [group] Num: group->Count())
            ->TakeIf(Num > 1)
            ->(Opp)
            ->Chain([ Name ])
            ->Sort()
            ->Concat("|")
    }

Show team name and division.

In [27]:
TeamsWithDiv->{ Name, XDiv }

Seq<{str,str}>
   0) { Name: ARI, XDiv: ARI|SEA|SF_|STL }
   1) { Name: ATL, XDiv: ATL|CAR|NO_|TB_ }
   2) { Name: BAL, XDiv: BAL|CIN|CLE|PIT }
   3) { Name: BUF, XDiv: BUF|MIA|NE_|NYJ }
   4) { Name: CAR, XDiv: ATL|CAR|NO_|TB_ }
   5) { Name: CHI, XDiv: CHI|DET|GB_|MIN }
   6) { Name: CIN, XDiv: BAL|CIN|CLE|PIT }
   7) { Name: CLE, XDiv: BAL|CIN|CLE|PIT }
   8) { Name: DAL, XDiv: DAL|NYG|PHI|WAS }
   9) { Name: DEN, XDiv: DEN|KC_|OAK|SD_ }
  10) { Name: DET, XDiv: CHI|DET|GB_|MIN }
  11) { Name: GB_, XDiv: CHI|DET|GB_|MIN }
  12) { Name: HOU, XDiv: HOU|IND|JAC|TEN }
  13) { Name: IND, XDiv: HOU|IND|JAC|TEN }
  14) { Name: JAC, XDiv: HOU|IND|JAC|TEN }
  15) { Name: KC_, XDiv: DEN|KC_|OAK|SD_ }
  16) { Name: MIA, XDiv: BUF|MIA|NE_|NYJ }
  17) { Name: MIN, XDiv: CHI|DET|GB_|MIN }
  18) { Name: NE_, XDiv: BUF|MIA|NE_|NYJ }
  19) { Name: NO_, XDiv: ATL|CAR|NO_|TB_ }
  20) { Name: NYG, XDiv: DAL|NYG|PHI|WAS }
  21) { Name: NYJ, XDiv: BUF|MIA|NE_|NYJ }
  22) { Name: OAK, XDiv: DEN|KC_|OAK|SD

Show distinct divisions.

In [28]:
TeamsWithDiv
    ->GroupBy(XDiv, [auto] XTeams)
    .XDiv

Seq<str>
   0) ARI|SEA|SF_|STL
   1) ATL|CAR|NO_|TB_
   2) BAL|CIN|CLE|PIT
   3) BUF|MIA|NE_|NYJ
   4) CHI|DET|GB_|MIN
   5) DAL|NYG|PHI|WAS
   6) DEN|KC_|OAK|SD_
   7) HOU|IND|JAC|TEN


Show distinct divisions in a simpler way.

In [29]:
TeamsWithDiv.XDiv->Distinct()

Seq<str>
   0) ARI|SEA|SF_|STL
   1) ATL|CAR|NO_|TB_
   2) BAL|CIN|CLE|PIT
   3) BUF|MIA|NE_|NYJ
   4) CHI|DET|GB_|MIN
   5) DAL|NYG|PHI|WAS
   6) DEN|KC_|OAK|SD_
   7) HOU|IND|JAC|TEN


Make a `Divisions` table with some statistics.

In [30]:
Divisions :=
    TeamsWithDiv
    ->GroupBy(
        [key] Div: XDiv,
        [group] Wins: group->Sum(Wins),
        [group] TotAP: group->Sum(TotAP),
        [group] TotBP: group->Sum(TotBP),
        [group] TotDiff: group->Sum(TotDiff),
        [auto] XTeams)

Show the type. Note that the nested `XTeams` tables include a further nested `XGames` table.

In [31]:
Divisions->GetType()

{Div:s, TotAP:i8, TotBP:i8, TotDiff:i8, Wins:i8, XTeams:{Name:s, Number:i8, TotAP:i8, TotBP:i8, TotDiff:i8, Wins:i8, XGames:{AP:i8, B:s, BP:i8, D:d, W:i8}*}*}*


Show the division statistics, without the nested `XTeams` tables.

In [32]:
Divisions
    +>{ XTeams: null }
    ->Sort(Wins, TotDiff, Div)

Seq<{str,i8,i8,i8,i8}>
   0) { Div: BUF|MIA|NE_|NYJ, TotAP: 1431, TotBP: 1375, TotDiff: 56, Wins: 36 }
   1) { Div: ATL|CAR|NO_|TB_, TotAP: 1335, TotBP: 1321, TotDiff: 14, Wins: 36 }
   2) { Div: CHI|DET|GB_|MIN, TotAP: 1365, TotBP: 1233, TotDiff: 132, Wins: 33 }
   3) { Div: BAL|CIN|CLE|PIT, TotAP: 1325, TotBP: 1229, TotDiff: 96, Wins: 33 }
   4) { Div: DAL|NYG|PHI|WAS, TotAP: 1529, TotBP: 1537, TotDiff: -8, Wins: 32 }
   5) { Div: DEN|KC_|OAK|SD_, TotAP: 1561, TotBP: 1490, TotDiff: 71, Wins: 31 }
   6) { Div: HOU|IND|JAC|TEN, TotAP: 1534, TotBP: 1573, TotDiff: -39, Wins: 30 }
   7) { Div: ARI|SEA|SF_|STL, TotAP: 1193, TotBP: 1515, TotDiff: -322, Wins: 25 }


## Further Exploration

Possible further exploration could include:
* What divisions played each other?
* Were there some division pairs that had many more games than other pairs?
* Is there structure among divisions? That is, just as each team belongs to a clique of four teams, called its division, is there a grouping of divisions?